In [1]:
import lightkurve as lk
import pandas
import pathlib
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import interact
import ipywidgets as widget
import pandas
import statistics as st

#from google.colab import files
#%matplotlib qt5

print("Imports done.")

Imports done.


In [4]:
import numpy as np
def outlier_removal(lc, sigma_value):
    lc = lc.remove_outliers(sigma = sigma_value)
    return lc #removes outliers

def plot_EPIC_ID(epic, bitmask, remove_outlier = True, sigma_value = 2.5):
    pixelfile = lk.search_targetpixelfile(epic).download(quality_bitmask=bitmask) #gets the data about the EPIC ID
    #fluxes = pixelfile.flux
    #print(np.std(fluxes[154]))
    #print(fluxes)
    
    #pixelfile.plot(frame = 128)
    #pixelfile.plot(frame = 0)
    fname = str(epic)+'.fits'
               
    lc = pixelfile.to_lightcurve(aperture_mask='all') #gets the light curve information
    if remove_outlier == True:
        lc = outlier_removal(lc, sigma_value) #calls the function that removes outliers
        print("Outliers removed")
    else:
        print("Outliers remain")
    lc.to_fits(path=fname, overwrite=True)
    #files.download(fname)    
    return lc, pixelfile #returns the light curve information
def epic_plotter(targets, bitmask_qual, remove_outlier):
    flux = []
    time = []
    
    # gets the light curve information for the ith EPIC ID in targets
    light_curve_info, pxfl = plot_EPIC_ID(targets, bitmask_qual, remove_outlier) 
    
    
    # gets the flux data about the ith EPIC ID, and adds it to the list of information 
    # about the fluxes of all the EPIC ID's
    flux.append(light_curve_info.flux) 
    
    # gets the time data about the ith EPIC ID, and adds it to the list of information 
    # about the times of all the EPIC ID's
    time.append(light_curve_info.time) 
   
    # creates a dataframe with the time and flux
    data = pandas.DataFrame({"Time":time[-1], "Flux":flux[-1]}) 
    
    # sets the time as the index for finding the flux
    data.set_index(data.columns[0], inplace=True, drop = True) 
   

    # gets the flux data about the EPIC ID
    
    
    flux = list(data["Flux"])
    
    total = len(flux)
    start = 0
    end = total
    
    flux = flux[start:end]
    median = st.median(flux)
    for i in flux:
       i = i/median
    
    
    # gets the time information about the EPIC ID
    time = list(data.index) 
    time = time[start:end]
    
    sz =[10,8]
    scale=1.3
    
    # plots using matplotlib
    #plt.gcf().set_size_inches(sz[0]*1.2,sz[1]/2.6)
    plt.plot(time, flux) 
    
    
    # light_curve_info.plot() #plots using the lightkurve.plot function
    pxfl.interact()
    
    plt.show() 
    
    #get cody graph from the paper
    img = '.\\..\\cody_graphs\\' +str(targets) + '.png'
    # print(img)  
    
    #plt.gcf().set_size_inches(sz[0]*scale,sz[1]*scale)
    plt.imshow(mpimg.imread(img))
    print('total points: ', total)
    print('data pts: ', len(flux), 'Start: ', start, 'End: ', end)
    print('Mean: ', st.mean(flux))
    print('Median: ', st.median(flux))
    print('Variance: ', st.variance(flux))

print("Functions defined.")

Functions defined.


In [5]:
base = 'https://raw.githubusercontent.com/SparshJohri/Kepler_Data_Analysis/master/'
file = 'Campaign_2_EPICS.csv'
campaign_epics = pandas.read_csv(base+file)

file = "CodyTable.csv"
campaign_epics = list(campaign_epics[campaign_epics.columns[0]])
cody_epics = pandas.read_csv(base+file)

cody_epics = list(cody_epics[cody_epics.columns[0]])


target_list = cody_epics
interact(epic_plotter, targets = target_list, bitmask_qual = ["default","hard", "none", "hardest"], \
         remove_outlier = [False, True])

interactive(children=(Dropdown(description='targets', options=(203382255, 203725791, 203786695, 203789507, 203…

<function __main__.epic_plotter(targets, bitmask_qual, remove_outlier)>